In [107]:
from hw6_code_that_we_use import parse_observations, unsupervised_HMM, sample_sentence
from nltk.corpus import cmudict
import re
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random

In [116]:
def sample_sentence(hmm, obs_map, n_words=100, seed=None):
    # Get reverse map.
    obs_map_r = obs_map_reverser(obs_map)

    # Sample and convert sentence.
    emission, states = hmm.generate_emission(n_words, seed=seed)
    sentence = [if(obs_map_r[i]!=' '): obs_map_r[i] for i in emission]


    return ' '.join(sentence).capitalize()

SyntaxError: invalid syntax (955452716.py, line 7)

In [108]:
import nltk
nltk.download('cmudict')

# constants
SYLLABLE_DICTIONARY_PATH = 'data/Syllable_dictionary.txt'
SHAKESPEARE_DATA = 'data/shakespeare.txt'

[nltk_data] Downloading package cmudict to /Users/acheron/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


In [109]:
syllable_dict = {}
with open(SYLLABLE_DICTIONARY_PATH, 'r') as f:
    for line in f:
        line = line.strip()

        if line:
            sentence = line.split(' ')
            word = sentence[0]
            syllable_dict[word] = sentence[1:]

In [110]:
def has_digit(string):
    return any(char.isdigit() for char in string)


def remove_punctuation(string):
    return re.sub(r'[^\w\s]', '', string)

with open(SHAKESPEARE_DATA, 'r') as f:
    all_data = f.readlines()
    all_lines = list(filter(lambda line: not has_digit(line), all_data))

    def clean_line(line):
        words = line.split(' ')

        words_no_punc = []

        for word in words:
            # Remove punctuation
            words_no_punc.append(remove_punctuation(word).strip())

        return words_no_punc

    X = list(map(clean_line, all_lines))

    X, X_map = parse_observations(X)

In [111]:
num_hidden_states = 10
# more iters take a while
N_iters = 10
HMM = unsupervised_HMM(X, num_hidden_states, N_iters, seed=None)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:37<00:00,  3.72s/it]


In [112]:
def convert(lst):
    return ([i for i in lst.split()])

def sentenceSyllable(sent): #for haikus only
    sentence = convert(sent)
    syllables = 0
    for word in sentence:
        word = remove_punctuation(word)
        if(word != ' '):  
            if (word.lower() in syllable_dict) == False:
                return 1000 #auto reject
            
            count = syllable_dict[word.lower()][0]
            
            
            if 'E' in count:
                count = int(count[1:])
            else:
                count = int(count)
        else:
            count = 0
        
            
        syllables += count
    
    return syllables

In [113]:
def ShakespeareHaiku():
    #print("Running: 0%", end='\r')
    haiku = []
    five = True
    while five:
        test = sample_sentence(HMM, X_map, n_words=random.randint(1, 5))
        if sentenceSyllable(test) == 5:
            five = False
            haiku.append(test[:-3])
    #print("Running: 33%", end='\r')
    
    seven = True
    while seven:
        test = sample_sentence(HMM, X_map, n_words=random.randint(1, 7))
        if sentenceSyllable(test) == 7:
            seven = False
            haiku.append(test[:-3])            
    #print("Running: 67%", end='\r')
    
    five = True
    while five:
        test = sample_sentence(HMM, X_map, n_words=random.randint(1, 5))
        if sentenceSyllable(test) == 5:
            five = False
            haiku.append(test[:-3]+".")
    #print("Done                    ", end='\r')
    #print()
    #print()
    
    for i in range(3):
        print(haiku[i])
    
    return haiku

In [115]:
ShakespeareHaiku()

Eye love worse to thee
 to which old defects might o
The to fickle were.


['Eye love worse to thee',
 ' to which old defects might o',
 'The to fickle were.']